# Stable Diffusion 深度剖析

Stable Diffusion 是一款强大的文本生成图像模型。市面上有多种网站和工具可帮助你轻松使用它。该模型也已[集成进 Hugging Face 的 diffusers 库](https://huggingface.co/blog/stable_diffusion)，只需几行代码即可生成图像：

```python
from diffusers import StableDiffusionPipeline
pipe = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1-base",
    variant="fp16",
    torch_dtype=torch.float16,
).to("cuda")
image = pipe("An astronaut scuba diving").images[0]
```

在本笔记本中，我们将深入剖析这些易用接口背后的代码，了解其内部工作原理。首先，我们会把上面的简洁示例“拆解”成一段看似可怕的大块代码；接着，我们将逐一检视不同的组件，弄清它们的职责与作用。等到笔记本结束时，你应该能够随心所欲地调整与改写。


In [ ]:
from base64 import b64encode

import numpy
import torch
from diffusers import AutoencoderKL, LMSDiscreteScheduler, UNet2DConditionModel
from huggingface_hub import notebook_login

# For video display:
from IPython.display import HTML
from matplotlib import pyplot as plt
from pathlib import Path
from PIL import Image
from torch import autocast
from torchvision import transforms as tfms
from tqdm.auto import tqdm
from transformers import CLIPTextModel, CLIPTokenizer, logging
import os

torch.manual_seed(1)
if not (Path.home()/'.cache/huggingface'/'token').exists(): notebook_login()

# 忽略加载 CLIPTextModel 时的多余警告
logging.set_verbosity_error()

# 根据硬件环境选择设备：优先使用 CUDA，其次 MPS，否则 CPU
torch_device = "cuda" if torch.cuda.is_available() \
    else "mps" if torch.backends.mps.is_available() \
    else "cpu"
# 如果是 MPS 设备，则开启后备机制以兼容更多操作
if torch_device == "mps":
    os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = "1"

## 加载模型

以下代码（以及下一节的代码）修改自 [Hugging Face 示例笔记本](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb)。

它将下载并设置我们将使用的相关模型和组件。现在先直接运行，然后移动到下一节，确认一切正常后再深入学习细节。

如果你已经加载了 pipeline，也可以通过 `pipe.unet`、`pipe.vae` 等方式访问其中的组件。

在本笔记本中，我们并未使用任何显存优化技巧——如果你的 GPU 显存不足，可以查看 pipeline 代码，参考诸如 **attention slicing**、切换到半精度（fp16）、将 VAE 留在 CPU 等方法来节省内存。


In [ ]:
# 加载用于将潜变量解码为图像空间的自编码器模型
vae = AutoencoderKL.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    subfolder="vae"
)

# 加载分词器和文本编码器，用于将文本进行分词和编码
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

# 加载用于生成潜变量的 UNet 条件模型
unet = UNet2DConditionModel.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    subfolder="unet"
)

# 配置噪声调度器，使用 LMS 离散调度器
scheduler = LMSDiscreteScheduler(
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    num_train_timesteps=1000
)

# 将模型移动到 GPU（或指定的设备）
vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device)


# TODO 待完成